In [78]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

In [2]:
vgg16 = models.vgg16(pretrained=True)

In [37]:
def conv_backend(in_channels, out_channels):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=2, dilation=2)

In [38]:
class CSRNet(nn.Module):
    
    def __init__(self):
        super(CSRNet, self).__init__()
        features = list(vgg16.features)[:23]
        self.features = nn.ModuleList(features).eval()
        self.conv1 = conv_backend(512, 512)
        self.conv2 = conv_backend(512, 512)
        self.conv3 = conv_backend(512, 512)
        self.conv4 = conv_backend(512, 256)
        self.conv5 = conv_backend(256, 128)
        self.conv6 = conv_backend(128, 64)
        self.convfinal = nn.Conv2d(64, 1, kernel_size=1, stride=1, padding=0, dilation=1)
        
    def forward(self,x):
        for model in self.features:
            x = model(x)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.convfinal(x))
        return x

In [39]:
model = CSRNet()
use_gpu = torch.cuda.is_available()
if use_gpu:
    model = model.cuda()

In [40]:
model

CSRNet(
  (features): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Con

In [93]:
params = list(model.parameters())
print(len(params))

34


In [94]:
nb_params = 0
for layer in params:
    n = 1
    for s in list(layer.size()):
        n *= s
    nb_params += n
nb_params
# on a bien les 16.26M

16263489

In [45]:
input = torch.randn(2, 3, 224, 224)
out = model(input)
print(out)

tensor([[[[0.1148, 0.1146, 0.1143,  ..., 0.1147, 0.1128, 0.1133],
          [0.1148, 0.1148, 0.1141,  ..., 0.1139, 0.1122, 0.1124],
          [0.1123, 0.1130, 0.1129,  ..., 0.1123, 0.1124, 0.1130],
          ...,
          [0.1130, 0.1134, 0.1121,  ..., 0.1137, 0.1129, 0.1127],
          [0.1126, 0.1120, 0.1110,  ..., 0.1120, 0.1136, 0.1140],
          [0.1117, 0.1118, 0.1107,  ..., 0.1120, 0.1140, 0.1144]]],


        [[[0.1152, 0.1151, 0.1144,  ..., 0.1150, 0.1127, 0.1133],
          [0.1152, 0.1153, 0.1148,  ..., 0.1138, 0.1117, 0.1127],
          [0.1125, 0.1137, 0.1132,  ..., 0.1129, 0.1132, 0.1136],
          ...,
          [0.1130, 0.1131, 0.1117,  ..., 0.1134, 0.1123, 0.1119],
          [0.1121, 0.1123, 0.1110,  ..., 0.1123, 0.1138, 0.1139],
          [0.1130, 0.1118, 0.1098,  ..., 0.1120, 0.1146, 0.1139]]]],
       grad_fn=<ReluBackward0>)


In [46]:
out.size()

torch.Size([2, 1, 28, 28])

In [81]:
out.view(2,-1).sum(1).detach().numpy()

array([88.275314, 88.249596], dtype=float32)

In [ ]:
def train(model,data_loader,loss_fn,optimizer,n_epochs=1):
    
    model.train(True)
    
    loss_train = np.zeros(n_epochs)
    acc_train = np.zeros(n_epochs)
    
    for epoch_num in range(n_epochs):
        running_corrects = 0.0
        running_loss = 0.0
        size = 0

        for data in data_loader:
            inputs, labels = data
            bs = labels.size(0)
            
            if use_gpu:
                inputs.cuda()
            outputs = model(inputs)
            #loss = np.mean(np.abs(outputs.view(bs,-1).sum(1).detach().numpy() - labels))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss
            running_corrects += ?
            size += bs
        epoch_loss = running_loss / size
        epoch_acc = running_corrects.item() / size
        loss_train[epoch_num] = epoch_loss
        acc_train[epoch_num] = epoch_acc
        print('Train - Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        
    return loss_train, acc_train

In [ ]:
model = CSRNet()
loss = 
learning_rate = 1e-6
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
l_t, a_t = train(model, train_loader, loss, optimizer, n_epochs=10)

In [ ]:
def test(model,data_loader):
    model.train(False)

    running_corrects = 0.0
    running_loss = 0.0
    size = 0
    
    with torch.no_grad():
        for data in data_loader:
            inputs, labels = data    
            bs = labels.size(0)

            if use_gpu:
                inputs.cuda()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss
            running_corrects += ?
            size += bs

    print('Test - Loss: {:.4f} Acc: {:.4f}'.format(running_loss / size, running_corrects.item() / size))